In [1]:
import folium
import json
import geopandas as gpd
from folium import plugins

# Load GeoJSON file
with open('text2ovq_case_study.geojson', 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Extract all coordinates to calculate map center
all_coords = []
for feature in geojson_data['features']:
    geom = feature['geometry']
    if geom['type'] == 'Point':
        all_coords.append(geom['coordinates'])
    elif geom['type'] == 'Polygon':
        # For polygons, take all coordinates from the first ring
        for coord in geom['coordinates'][0]:
            all_coords.append(coord)

# Calculate center point
if all_coords:
    center_lon = sum(coord[0] for coord in all_coords) / len(all_coords)
    center_lat = sum(coord[1] for coord in all_coords) / len(all_coords)
else:
    center_lon, center_lat = 0, 0

print(f"Map center: longitude {center_lon:.6f}, latitude {center_lat:.6f}")
print(f"Total features: {len(geojson_data['features'])}")


Map center: longitude 13.186364, latitude 45.609135
Total features: 17


In [ ]:
# Create base map - auto-fit to all data points
# m = folium.Map(tiles='OpenStreetMap')
m = folium.Map(tiles='cartodb positron')

# Calculate bounding box to ensure all points are visible
min_lat, max_lat = float('inf'), float('-inf')
min_lon, max_lon = float('inf'), float('-inf')

for coord in all_coords:
    lon, lat = coord
    min_lat = min(min_lat, lat)
    max_lat = max(max_lat, lat)
    min_lon = min(min_lon, lon)
    max_lon = max(max_lon, lon)

# Set map bounds to display all data
southwest = [min_lat - 0.03, min_lon - 0.01]  # More margin at bottom
northeast = [max_lat + 0.005, max_lon + 0.01]  # Less margin at top
m.fit_bounds([southwest, northeast])

# Process all data, display as marker points
for feature in geojson_data['features']:
    geom = feature['geometry']
    
    if geom['type'] == 'Point':
        lon, lat = geom['coordinates']
        # Display Point with blue marker
        folium.Marker(
            location=[lat, lon],
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)
        
    elif geom['type'] == 'Polygon':
        coords = geom['coordinates'][0]
        folium_coords = [(lat, lon) for lon, lat in coords]
        
        # Calculate polygon center point
        center_lat = sum(lat for lat, lon in folium_coords) / len(folium_coords)
        center_lon = sum(lon for lat, lon in folium_coords) / len(folium_coords)
        
        # Display Polygon with blue marker
        folium.Marker(
            location=[center_lat, center_lon],
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)

# Display map
m


In [3]:
# Save map as HTML file
m.save('text2ovq_case_study.html')

print("Map saved as text2ovq_case_study.html")


Map saved as text2ovq_case_study.html
